In [149]:
import json
import os
import pandas as pd
from genData import load_model

import json
import os
import pandas as pd
from genData import load_model
import re
import numpy as np
import pickle

from importlib import reload
import mainlib
reload(mainlib)
import A_base as ab
reload(ab)
from  mainlib import fasorial, lista, sdict, mm, mm_sep


import collections
import of_proc
reload(of_proc)
from of_proc import *

# Informações

In [2]:
def read_info(path = '../info/info.json'):
    global PATH, OUTPUT_PATH, PROCESSED_PATH, PERFIS_PATH, HORARIOS_PATH, PATH_MM, PATH_FAS, \
    DEG, MOR, PER, FAS, OUTPUT_HEADER,FAS_CONCAT, MM_CONCAT, PICKLES_CONCAT, OUT_INDEX
    with open(path) as f:
        info = json.load(f)
        PATH = info['paths']['fasoriais']['input']
        OUTPUT_PATH = info['paths']['fasoriais']['output']
        PROCESSED_PATH = info['paths']['fasoriais']['processed']
        PERFIS_PATH = info['paths']['perfis']['perfis']
        HORARIOS_PATH = info['paths']['perfis']['horarios']
        PATH_MM = info['paths']['mm']['input']
        PATH_FAS = info['paths']['fasoriais']['input']
        DEG = info['paths']['nn_models']['degrau']
        MOR = info['paths']['nn_models']['morro']
        PER = info['paths']['nn_models']['perfil']
        FAS = info['paths']['nn_models']['fasorial']
        FAS_CONCAT =  info['paths']['concat']['fas']
        MM_CONCAT =  info['paths']['concat']['mm'] 
        PICKLES_CONCAT=  info['paths']['concat']['pickles']        
        OUTPUT_HEADER = info['output']['header']
        OUT_INDEX = info['output']['header']

In [3]:
read_info()

In [4]:
OUT_CONCAT = '../outputs/concat/'
OUT = '../outputs/all/'

In [5]:
#preguiça -- arrumar
out_index = OUT_INDEX

### Carregar modelos neurais

# Coleções principais

#### Lista principal

In [6]:
global LISTA_CAD

#### Modelos neurais

In [7]:
global PERFIL, DEGRAU, MORRO, NN_FAS
PERFIL, DEGRAU, MORRO, NN_FAS = load_models(PER, DEG, MOR, FAS)

Instructions for updating:
Colocations handled automatically by placer.


#### Arquivos

In [8]:
global files_fas, files_mm

#### Perfis e horários

In [9]:
global HORARIOS, PERFIS, ATIVS, DPERFIS, DPERFIS_NORMAL, DPERFIS_TROCA

#### Estruturas auxiliares

In [10]:
global dim, dmi, df_fas, dfiles_mm, dfiles_fas_troca, dfiles_mm_troca, dfiles_fas_normal, dfiles_mm_normal
global o_i_normal, o_i_troca, o_instas, list_i_normal, list_i_troca, list_intas

## Lista de medidores e lista por horários

In [11]:
LISTA_CAD = get_list()

In [12]:
HORARIOS, PERFIS, ATIVS = get_horario_info()

### Dicionários e listas

In [13]:
dim, dmi = dicts(LISTA_CAD)

In [14]:
#gls i_normal -> instalações normal
# i_troca -> instalações com troca
#o_i_normal -> ordenado instalações normal
o_i_normal = collections.OrderedDict(sorted(LISTA_CAD.get_users(index = 'instalacao', troca=0,tipo='dictionary').items()))
o_i_troca = collections.OrderedDict(sorted(LISTA_CAD.get_users(index = 'instalacao', troca=1,tipo='dictionary').items()))
o_instas =  collections.OrderedDict(sorted({**o_i_normal, **o_i_troca}.items()))

##### Dicionário com os horário de funcionamento de cada cliente

In [15]:
DPERFIS = sdict()
for i in PERFIS.index:
    j = PERFIS.loc[i,'Ramo de Atividade']
    if HORARIOS.index.contains(j):
        DPERFIS[i] = list(HORARIOS.loc[j,:])
    else:
        DPERFIS[i] = list(HORARIOS.iloc[0,:])

DPERFIS_NORMAL={}
for i in DPERFIS:
    if i in o_i_normal:
        DPERFIS_NORMAL[i] = o_i_normal[i]

DPERFIS_TROCA={}
for i in DPERFIS:
    if i in o_i_troca:
        DPERFIS_TROCA[i] = o_i_troca[i]

In [16]:
dfiles_fas, nf_fas=get_files_3(list(o_i_normal.keys()), dim,  PATH_FAS)
dfiles_mm, nf_mm = get_files_3(list(o_i_normal.keys()),dim , PATH_MM)

In [17]:
dfiles_fas2 = deslistar(dfiles_fas)
dilfes_mm2 = deslistar(dfiles_mm)

In [18]:
dfiles_fas_concat, nf_fas_concat = get_files_2( list(o_i_troca.keys()), FAS_CONCAT)
dfiles_mm_concat, nf_mm_concat = get_files_2( list(o_i_troca.keys()), MM_CONCAT)

## l_______________________________________________________________________

In [19]:
dfiles_fas_concat = get_files_troca(dfiles_fas_concat)

In [20]:
dfiles_mm_concat= get_files_troca(dfiles_mm_concat)

In [21]:
odf_concat = collections.OrderedDict(sorted(dfiles_fas_concat.items()))

In [22]:
odmm_concat = collections.OrderedDict(sorted(dfiles_mm_concat.items()))

## l_______________________________________________________________________

### Um exemplo com muitos valores faltantes

In [49]:
i1 = 'BT2A420415794'

In [50]:
LISTA_CAD.df[LISTA_CAD.df['instalacao']=='BT2A420415794']

,cliente,instalacao,medidor,data_inicio,data_fim,lote,ssn,dez,jan,fev,mar,abr,troca
1085,BT2A420415794,BT2A420415794,9103268,2016-10-24,2099-12-31,08*,89551180637000222270,2018-12-12,2019-01-17,2019-02-14,2019-03-18,2019-04-15,1-Sem troca de medidor


In [135]:
f1 = fasorial(name = i1 )

In [136]:
fl1 = ab.search_list('9103268', os.listdir(PATH_FAS))

In [137]:
df1 = f1.read_fasorial( os.path.join(PATH_FAS,fl1), dtype=None).copy()

In [152]:
ddays = o_i_normal['BT2A420415794']

In [157]:
ddays2 = {12: pd.Timestamp('2018-12-12 00:00:00'),
 1: pd.Timestamp('2019-01-17 00:00:00'),
 2: pd.Timestamp('2019-02-14 00:00:00'),
 3: pd.Timestamp('2019-03-18 00:00:00'),
 4: pd.Timestamp('2019-04-15 00:00:00')}

In [158]:
f1.split_months(ddays=ddays)

{2018: {},
 2019: {1: <mainlib.fasorial at 0x7f1b24818da0>,
  2: <mainlib.fasorial at 0x7f1b246af5c0>,
  3: <mainlib.fasorial at 0x7f1b24233668>,
  4: <mainlib.fasorial at 0x7f1b2469fda0>}}

In [165]:
for i in (1,2,3,4):
    f1.months[2019][i].proc_fasorial()

In [167]:
f1.months[2019][1].outputs

{'name': 'noname',
 'shape_no_proc': (8562, 39),
 'wrong_columns': ['ponto_medicao',
  'IA',
  'IB',
  'IC',
  'VA',
  'VB',
  'VC',
  'VAB',
  'VBC',
  'VAC',
  'PA',
  'PB',
  'PC'],
 'empty_values': {'IA': 0,
  'IB': 0,
  'IC': 0,
  'AIA': 0,
  'AIB': 0,
  'AIC': 0,
  'VA': 0,
  'VB': 0,
  'VC': 0,
  'AVA': 0,
  'AVB': 0,
  'AVC': 0,
  'VAB': 0,
  'VBC': 0,
  'VAC': 0,
  'AVAB': 0,
  'AVBC': 0,
  'AVAC': 0,
  'PA': 0,
  'PB': 0,
  'PC': 0,
  'FPA': 0,
  'FPB': 0,
  'FPC': 0},
 'missing_rows': 26,
 'shape_proc': (2880, 26)}

## l_______________________________________________________________________

# Análise de valores faltantes para MM

In [194]:
m1 = mm(name=i1)

In [196]:
m1.read_mm(os.path.join(PATH_MM, ab.search_list('9103268',os.listdir(PATH_MM))))

,cliente,valor
datetime,,
2018-12-01 01:00:00,09103268,-999.00
2018-12-01 02:00:00,09103268,-999.00
2018-12-01 03:00:00,09103268,-999.00
2018-12-01 04:00:00,09103268,-999.00
2018-12-01 05:00:00,09103268,-999.00
2018-12-01 06:00:00,09103268,-999.00
2018-12-01 07:00:00,09103268,-999.00
2018-12-01 08:00:00,09103268,-999.00
2018-12-01 09:00:00,09103268,-999.00


In [199]:
m1.split_months(ddays=ddays)

{2018: {},
 2019: {1: <mainlib.mm at 0x7f1b24ad1c88>,
  2: <mainlib.mm at 0x7f1b24ad8be0>,
  3: <mainlib.mm at 0x7f1b248ebcf8>,
  4: <mainlib.mm at 0x7f1b24ad8cc0>}}

In [201]:
for m in (1,2,3,4):
    m1.months[2019][m].proc_mm()

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [208]:
m1.months[2019][4].df

,valor
datetime,
2019-03-16 00:00:00,NaN
2019-03-16 01:00:00,NaN
2019-03-16 02:00:00,NaN
2019-03-16 03:00:00,NaN
2019-03-16 04:00:00,NaN
2019-03-16 05:00:00,NaN
2019-03-16 06:00:00,NaN
2019-03-16 07:00:00,NaN
2019-03-16 08:00:00,NaN


In [193]:
m1.proc_mm()

,valor
datetime,
2018-12-01 01:00:00,NaN
2018-12-01 02:00:00,NaN
2018-12-01 03:00:00,NaN
2018-12-01 04:00:00,NaN
2018-12-01 05:00:00,NaN
2018-12-01 06:00:00,NaN
2018-12-01 07:00:00,NaN
2018-12-01 08:00:00,NaN
2018-12-01 09:00:00,NaN


## l_______________________________________________________________________

In [97]:
string = ',39'

In [66]:
float((''.join(re.findall('[0-9,.]',string))).replace(',','.'))

0.39

In [111]:
def proc_string2(string, verbose = 0):
    if type(string) == str or type(string) == object or type(string) == 'O':
        try:
            return float((''.join(re.findall('[0-9,.]',string))).replace(',','.'))
        except Exception as e:
            return np.nan
    else:
        return 'X'

In [98]:
proc_string2(string)

0.39

## l_______________________________________________________________________

In [106]:
f1.df['IA'].dtypes

dtype('O')

In [58]:
df = df1.copy()

In [59]:
df.head()

,ponto_medicao,serial,RTC,RTP,KE,IA,IB,IC,AIA,AIB,...,DH,FREQ,SEQ,DEMAND_C1,DEMAND_C2,DEMAND_C3,TOT_C1,TOT_C2,TOT_C3,DEM_INTERVAL
data,,,,,,,,,,,,,,,,,,,,,
2018-12-01 00:00:05,BT2A420415794,9103268,1/1,1/1,1/1,0,",39",4,0,171,...,0,60,132,7,6,0,3905374,2123118,7116,15
2018-12-01 00:00:05,BT2A420415794,9103268,1/1,1/1,1/1,0,",39",4,0,171,...,0,60,132,7,6,0,3905374,2123118,7116,15
2018-12-01 00:15:03,BT2A420415794,9103268,1/1,1/1,1/1,0,",41","4,15",0,170,...,0,60,132,4,2,1,3905378,2123120,7117,15
2018-12-01 00:15:03,BT2A420415794,9103268,1/1,1/1,1/1,0,",41","4,15",0,170,...,0,60,132,4,2,1,3905378,2123120,7117,15
2018-12-01 00:15:03,BT2A420415794,9103268,1/1,1/1,1/1,0,",41","4,15",0,170,...,0,60,132,4,2,1,3905378,2123120,7117,15


In [125]:
f1.df.iloc[:,2:] = f1.df.iloc[:,2:].applymap(f1.proc_string)

In [126]:
f1.df

,ponto_medicao,serial,RTC,RTP,KE,IA,IB,IC,AIA,AIB,...,DH,FREQ,SEQ,DEMAND_C1,DEMAND_C2,DEMAND_C3,TOT_C1,TOT_C2,TOT_C3,DEM_INTERVAL
data,,,,,,,,,,,,,,,,,,,,,
2018-12-01 00:00:05,BT2A420415794,9103268,11.0,11.0,11.0,0.00,0.39,4.00,0,171,...,0,60,132,7,6,0,3905374,2123118,7116,15
2018-12-01 00:00:05,BT2A420415794,9103268,11.0,11.0,11.0,0.00,0.39,4.00,0,171,...,0,60,132,7,6,0,3905374,2123118,7116,15
2018-12-01 00:15:03,BT2A420415794,9103268,11.0,11.0,11.0,0.00,0.41,4.15,0,170,...,0,60,132,4,2,1,3905378,2123120,7117,15
2018-12-01 00:15:03,BT2A420415794,9103268,11.0,11.0,11.0,0.00,0.41,4.15,0,170,...,0,60,132,4,2,1,3905378,2123120,7117,15
2018-12-01 00:15:03,BT2A420415794,9103268,11.0,11.0,11.0,0.00,0.41,4.15,0,170,...,0,60,132,4,2,1,3905378,2123120,7117,15
2018-12-01 00:30:04,BT2A420415794,9103268,11.0,11.0,11.0,0.00,0.51,1.64,0,170,...,0,60,132,8,6,0,3905386,2123126,7117,15
2018-12-01 00:30:04,BT2A420415794,9103268,11.0,11.0,11.0,0.00,0.51,1.64,0,170,...,0,60,132,8,6,0,3905386,2123126,7117,15
2018-12-01 00:30:04,BT2A420415794,9103268,11.0,11.0,11.0,0.00,0.51,1.64,0,170,...,0,60,132,8,6,0,3905386,2123126,7117,15
2018-12-01 00:45:05,BT2A420415794,9103268,11.0,11.0,11.0,0.09,0.48,1.55,61,169,...,0,60,132,12,13,0,3905398,2123139,7117,15


In [64]:
f1.df.head()

,ponto_medicao,serial,RTC,RTP,KE,IA,IB,IC,AIA,AIB,...,DH,FREQ,SEQ,DEMAND_C1,DEMAND_C2,DEMAND_C3,TOT_C1,TOT_C2,TOT_C3,DEM_INTERVAL
data,,,,,,,,,,,,,,,,,,,,,
2018-12-01 00:00:05,BT2A420415794,9103268,11.0,11.0,11.0,0.0,0.39,4.00,0,171,...,0,60,132,7,6,0,3905374,2123118,7116,15
2018-12-01 00:00:05,BT2A420415794,9103268,11.0,11.0,11.0,0.0,0.39,4.00,0,171,...,0,60,132,7,6,0,3905374,2123118,7116,15
2018-12-01 00:15:03,BT2A420415794,9103268,11.0,11.0,11.0,0.0,0.41,4.15,0,170,...,0,60,132,4,2,1,3905378,2123120,7117,15
2018-12-01 00:15:03,BT2A420415794,9103268,11.0,11.0,11.0,0.0,0.41,4.15,0,170,...,0,60,132,4,2,1,3905378,2123120,7117,15
2018-12-01 00:15:03,BT2A420415794,9103268,11.0,11.0,11.0,0.0,0.41,4.15,0,170,...,0,60,132,4,2,1,3905378,2123120,7117,15


In [128]:
for c in f1.df.columns:
    print( c, f1.df[c].isna().sum())

ponto_medicao 0
serial 0
RTC 0
RTP 0
KE 0
IA 0
IB 0
IC 0
AIA 0
AIB 0
AIC 0
VA 0
VB 0
VC 0
AVA 0
AVB 0
AVC 0
VAB 0
VBC 0
VAC 0
AVAB 0
AVBC 0
AVAC 0
PA 0
PB 0
PC 0
FPA 0
FPB 0
FPC 0
DH 0
FREQ 0
SEQ 0
DEMAND_C1 0
DEMAND_C2 0
DEMAND_C3 0
TOT_C1 0
TOT_C2 0
TOT_C3 0
DEM_INTERVAL 0


In [ ]:
df.iloc[:,2:] = df.iloc[:,2:].applymap(self.proc_string)
outputs['empty_values'] = df.iloc[:,2:].isna().sum().to_dict()
df.fillna(value=self.dummy_val,inplace=True)   
df = df.resample('15min').first()
outputs['missing_rows'] = df.iloc[:,2].isna().sum()
df.fillna(value=self.dummy_val,inplace=True)
outputs['shape_proc'] = df.shape

## l_______________________________________________________________________

## l_______________________________________________________________________

# Importante

In [23]:
processados = []
erros = []
sem_hora = []
err_msg = {}

In [469]:
def loopy2(d_insta_to_days, d_insta_to_files_fas, d_insta_to_files_mm,PATH_FAS, PATH_MM, output_dir):
    df_out = pd.DataFrame(index=out_index) 
    global dfas, dmm
    dfas, dmm = sdict(), sdict()
    #processados = []
    erros = []
    sem_hora = []
    err_msg = {}
    #varre instalações
    c = 0
    c1 = 0
    for  instalacao in d_insta_to_days:
        c1 = c1 + 1        
        if (instalacao not in processados):
            medidor = dim[instalacao][0] if type(dim[instalacao]) is list else dim[instalacao]
            #try:
            if True:
                processados.append(instalacao)
                #carrega horário de funcionamento para o perfil atual
                try:
                    hours = list(HORARIOS.loc[ PERFIS.loc[instalacao,'Ramo de Atividade'], : ])
                except Exception as e:
                    hours = list(HORARIOS.iloc[0,:])
                    sem_hora.append(instalacao)
                    print('Erro 1: \n',e)    
                hours = [ h.strftime("%H:%M") for h in hours]

                #carrega dias de leitura
                ddays = {}
                ddays[12],ddays[1],ddays[2],ddays[3],ddays[4] = d_insta_to_days[instalacao][12], d_insta_to_days[instalacao][1], d_insta_to_days[instalacao][2], d_insta_to_days[instalacao][3], d_insta_to_days[instalacao][4]
                #ddays = d_med_days[int(medidor)]

                dfas = fasorial(name = instalacao, ddays=ddays)
                dfas.read_fasorial(os.path.join(PATH_FAS, d_insta_to_files_fas[instalacao]), dtype=None, sep = ',', index_col = [0])
                dfas.outputs['perfil'] = PERFIS.loc[instalacao,'Ramo de Atividade']

                dmm = mm(name = instalacao, ddays = ddays)
                dmm.read_mm(os.path.join(PATH_MM, d_insta_to_files_mm[instalacao]), skiprows=[0])
                
                proc_fas(dfas,ddays, NN_FAS, hours = hours)
                processa_mm(dmm, ddays, PERFIL, DEGRAU, MORRO)
                print('################################################')

                #Cria um dicionário de outputs por mês para o fasorial
                for y in dfas.months:
                    dfas.outputs[y] = {}
                    for month  in dfas.months[y]:
                        dfas.outputs[y][month] = dfas.months[y][month].outputs
                        #print(dfas[medidor].months[y][month].outputs)

                #diagnostico = ab.diag_series( get_sevs_mm(dmm[medidor]), get_sevs_fas(dfas[medidor]))
                df_out = cat_out2(df_out, dmm.outputs, dfas.outputs, ddays)
                #dt = df_out.T
                #dt.insert(loc = 15, column = 'Diag_final', value = ab.diag_series(dt['severidade-fasorial'], dt['severidade-perfil'] ) )
                #df_out = dt.T


                
                if c%100 == 0 or c > len(d_insta_to_days)-1:
                    print('Salvando csv!')
                    pickle.dump(processados, open('../pickles/all/processados.pkl','wb'))
                    pickle.dump(erros, open('../pickles/all/erros.pkl','wb'))
                    df_out.T.to_csv(output_dir + str(c-100) + '_to_' + str(c) +'.csv')
                    df_out=pd.DataFrame()
                print(c,c1, instalacao)
            #except Exception as e:
            else:
                erros.append(instalacao)
                err_msg[instalacao] = e
                print('Erro 2: \n',e)
            print('Salvando csv!')
            pickle.dump(processados, open('../pickles/all/processados.pkl','wb'))
            pickle.dump(erros, open('../pickles/all/erros.pkl','wb'))
            df_out.T.to_csv(output_dir + 'ultimos.csv')
            df_out=pd.DataFrame()



In [ ]:
loopy(o_i_troca, dfiles_fas_concat, dfiles_mm_concat,FAS_CONCAT, MM_CONCAT, OUT_CONCAT,sepmm=';')

In [ ]:
loopy(o_i_normal2, dfiles_fas2, dfiles_mm2,PATH_FAS, PATH_MM,OUT,sepmm=',')

## l_______________________________________________________________________